In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import scipy.integrate as integrate

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
sess = tf.InteractiveSession()

In [5]:
saver = tf.train.import_meta_graph('mnist/conv_99_new.meta')

In [6]:
saver.restore(sess,'mnist/conv_99_new')

INFO:tensorflow:Restoring parameters from mnist/conv_99_new


In [7]:
print(tf.global_variables())

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(5, 5, 32, 64) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'Variable_4:0' shape=(3136, 1024) dtype=float32_ref>, <tf.Variable 'Variable_5:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'Variable_6:0' shape=(1024, 10) dtype=float32_ref>, <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'Variable/Adam:0' shape=(5, 5, 1, 32) dtype=float32_ref>, <tf.Variable 'Variable/Adam_1:0' shape=(5, 5, 1, 32) dtype=float32_ref>, <tf.Variable 'Variable_1/Adam:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_1/Adam_1:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_2/Adam:0' shape=(5, 5, 32, 64) dtype=float32_ref>, <tf.Variable 'Variable_2

In [8]:
keep_prob = tf.get_default_graph().get_tensor_by_name("keep_prob:0")
x = tf.get_default_graph().get_tensor_by_name("x:0")
y_conv = tf.get_default_graph().get_tensor_by_name("y_conv:0")
y_conv_s = tf.get_default_graph().get_tensor_by_name("y_conv_s:0")

# Let's compute the geodesics..

In [9]:
def shift_pic(pic, shift=4):
    return np.roll(pic.reshape([28,28]), shift, axis=1).reshape([784,])

In [10]:
def create_vec_tensor(categories, target = y_conv_s):
    vec_tens = []
    for i in range(categories):
            vec_tens.append(tf.gradients(target[:,i], x)[0])
    return vec_tens

In [11]:
vec_tens = create_vec_tensor(10)
def compute_vec_grad(pic, vec_tens):
    var_grad = sess.run(vec_tens, feed_dict={x: pic.reshape([1,len(pic)]), keep_prob: 1.0})
    return np.asarray(var_grad).reshape([10, 784])

In [12]:
def create_vec_hessian(categories, target = y_conv_s):
    vec_hes = []
    for i in range(categories):
            vec_hes.append(tf.hessians(target[:,i], x)[0])
    return vec_hes

In [13]:
vec_hes = create_vec_hessian(10)
def compute_vec_hessian(pic, vec_hes):
    var_hessian = sess.run(vec_hes, feed_dict={x: pic.reshape([1,len(pic)]), keep_prob: 1.0})
    return np.asarray(var_hessian).reshape([784, 784, 10])

In [14]:
def compute_metric(pic, compute_vec_grad=compute_vec_grad, alpha=0.01):
    vec_grad = compute_vec_grad(pic, vec_tens)
    #vec_grad = vec_grad/np.sqrt(np.sum(vec_grad**2, axis=1)).reshape([10,1])
    return alpha*np.identity(784) + np.dot(np.transpose(vec_grad),vec_grad)

In [15]:
def compute_grad_metric(pic, compute_vec_grad=compute_vec_grad,
                        compute_vec_hessian=compute_vec_hessian):
    vec_grad = compute_vec_grad(pic, vec_tens)
    #vec_grad = vec_grad/np.sqrt(np.sum(vec_grad**2, axis=1)).reshape([10,1])
    vec_hessian = compute_vec_hessian(pic, vec_hes)
    hess_mult_grad = np.dot(vec_hessian, vec_grad)
    #still need to symmetrize, computes grad_k g^{ij} (first axis is grad, second 2 are g)
    return hess_mult_grad + np.swapaxes(hess_mult_grad, 1, 2)

In [16]:
def point_shift_integrand(start_pic, end_pic, t, metric=compute_metric, grad_metric=compute_grad_metric):
    curr_pic = (1-t)*start_pic + t*end_pic 
    numerator = np.dot(2*metric(curr_pic), end_pic-start_pic) + t*np.dot(np.dot(grad_metric(curr_pic),end_pic-start_pic), end_pic-start_pic)
    denominator = 2*np.sqrt( np.dot( np.dot(end_pic-start_pic, metric(curr_pic)), end_pic-start_pic))
    return numerator/denominator

In [17]:
def segment_shift_integrand(midpoint, leftpoint, rightpoint, t): 
    return point_shift_integrand(start_pic=leftpoint, end_pic=midpoint, t=t) + point_shift_integrand(start_pic=midpoint, end_pic=rightpoint, t=t)

In [18]:
def compute_point_shift_grad(midpoint, leftpoint, rightpoint):
    acc = []
    for pix in range(784):
        print("cur pixel: ", pix)
        acc.append(integrate.romberg(lambda t: segment_shift_integrand(midpoint, leftpoint, rightpoint, t)[pix], 0, 1, tol=1, rtol=1e-3))
    return np.asarray(acc)
    #return integrate.romberg(lambda t: segment_shift_integrand(midpoint, leftpoint, rightpoint, t), 0, 1)
    #return segment_shift_integrand(midpoint, leftpoint, rightpoint, 0)
    

In [19]:
def minimize_length_step(point_index, geod, r):
    grad_L = compute_point_shift_grad(geod[point_index,:], geod[point_index-1,:], geod[point_index+1,:])
    return geod[point_index] - r*grad_L 

In [20]:
#returns the geodesic with first index being the pt index and second being the actual point (or picture in the mnist case)
def divide(startpoint, endpoint, num_div):
    pt_divs = np.linspace(0, 1, num=num_div)

    geod = np.ones(len(pt_divs)).reshape([len(pt_divs),1])*startpoint.reshape([1,len(startpoint)]) + pt_divs.reshape([len(pt_divs),1])*(endpoint-startpoint).reshape([1,len(startpoint)])
    return pt_divs, geod

In [21]:
def compute_geodesic(startpoint, endpoint, num_div, r):
    grid, geod = divide(startpoint, endpoint, num_div)
    for pt_index in range(len(grid))[1:-1]:
        geod[pt_index] = minimize_length_step(pt_index, geod, r)
    return geod

In [22]:
start_pic = shift_pic(mnist.train.images[7],shift=-2)
end_pic = mnist.train.images[7]

In [23]:
g = compute_geodesic(start_pic, end_pic, 4, 0.1)

cur pixel:  0
cur pixel:  1
cur pixel:  2
cur pixel:  3
cur pixel:  4
cur pixel:  5
cur pixel:  6
cur pixel:  7
cur pixel:  8
cur pixel:  9
cur pixel:  10
cur pixel:  11
cur pixel:  12
cur pixel:  13
cur pixel:  14
cur pixel:  15
cur pixel:  16
cur pixel:  17
cur pixel:  18
cur pixel:  19
cur pixel:  20
cur pixel:  21
cur pixel:  22
cur pixel:  23
cur pixel:  24
cur pixel:  25
cur pixel:  26
cur pixel:  27
cur pixel:  28
cur pixel:  29
cur pixel:  30
cur pixel:  31
cur pixel:  32
cur pixel:  33
cur pixel:  34
cur pixel:  35
cur pixel:  36
cur pixel:  37
cur pixel:  38
cur pixel:  39
cur pixel:  40
cur pixel:  41
cur pixel:  42
cur pixel:  43
cur pixel:  44
cur pixel:  45
cur pixel:  46
cur pixel:  47
cur pixel:  48
cur pixel:  49
cur pixel:  50
cur pixel:  51
cur pixel:  52
cur pixel:  53
cur pixel:  54
cur pixel:  55
cur pixel:  56
cur pixel:  57
cur pixel:  58
cur pixel:  59
cur pixel:  60
cur pixel:  61
cur pixel:  62
cur pixel:  63
cur pixel:  64
cur pixel:  65
cur pixel:  66
cur p

KeyboardInterrupt: 

In [ ]:
g.shape